# 1. Dataset
## 1.1 Database Features


| Attribute              | Description                                                                 |
|------------------------|-----------------------------------------------------------------------------|
| A - fighter1           | Name of the first fighter.                                                  |
| B - fighter2           | Name of the second fighter.                                                 |
| C - event              | Name of the UFC event where the fight took place.                           |
| D - fight_outcome      | Indicates which fighter won the fight.                                      |
| E - origin_fight_url   | URL linking to the original source of the fight details.                    |
| F - fighter1_Name      | Full name of fighter 1.                                                     |
| G - fighter1_Nickname  | Nickname of fighter 1.                                                      |
| H - fighter1_Record    | Fight record (wins-losses-draws) of fighter 1.                              |
| I - fighter1_Height    | Height of fighter 1.                                                        |
| J - fighter1_Weight    | Weight of fighter 1.                                                        |
| K - fighter1_Reach     | Reach of fighter 1.                                                         |
| L - fighter1_Stance    | Fighting stance of fighter 1 (e.g., Orthodox, Southpaw).                    |
| M - fighter1_DOB       | Date of birth of fighter 1.                                                 |
| N - fighter1_SLpM      | Significant strikes landed per minute by fighter 1.                         |
| O - fighter1_StrAcc    | Striking accuracy of fighter 1.                                             |
| P - fighter1_SApM      | Significant strikes absorbed per minute by fighter 1.                       |
| Q - fighter1_StrDef    | Striking defense percentage of fighter 1.                                   |
| R - fighter1_TDAvg     | Average takedowns per 15 minutes for fighter 1.                             |
| S - fighter1_TDAcc     | Takedown accuracy of fighter 1.                                             |
| T - fighter1_TDDef     | Takedown defense percentage of fighter 1.                                   |
| U - fighter1_SubAvg    | Average number of submissions attempted per 15 minutes by fighter 1.        |
| V - fighter2_Name      | Full name of fighter 2.                                                     |
| W - fighter2_Nickname  | Nickname of fighter 2.                                                      |
| X - fighter2_Record    | Fight record (wins-losses-draws) of fighter 2.                              |
| Y - fighter2_Height    | Height of fighter 2.                                                        |
| Z - fighter2_Weight    | Weight of fighter 2.                                                        |
| AA - fighter2_Reach    | Reach of fighter 2.                                                         |
| AB - fighter2_Stance   | Fighting stance of fighter 2 (e.g., Orthodox, Southpaw).                    |
| AC - fighter2_DOB      | Date of birth of fighter 2.                                                 |
| AD - fighter2_SLpM     | Significant strikes landed per minute by fighter 2.                         |
| AE - fighter2_StrAcc   | Striking accuracy of fighter 2.                                             |
| AF - fighter2_SApM     | Significant strikes absorbed per minute by fighter 2.                       |
| AG - fighter2_StrDef   | Striking defense percentage of fighter 2.                                   |
| AH - fighter2_TDAvg    | Average takedowns per 15 minutes for fighter 2.                             |
| AI - fighter2_TDAcc    | Takedown accuracy of fighter 2.                                             |
| AJ - fighter2_TDDef    | Takedown defense percentage of fighter 2.                                   |
| AK - fighter2_SubAvg   | Average number of submissions attempted per 15 minutes by fighter 2.        |


In [15]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# (Re-fit your model if needed—skip these steps if `model` already exists)
df = pd.read_csv('../Data/ufc_fights_cleaned.csv')
for st in ['Open stance','Orthodox','Southpaw','Switch']:
    df[f'diff_Stance_{st.replace(" ","_")}'] = (
        df[f'fighter1_Stance_{st}'].astype(int)
      - df[f'fighter2_Stance_{st}'].astype(int)
    )
diff_cols = [c for c in df.columns if c.startswith('diff_')]
X = sm.add_constant(df[diff_cols])
y = df['fight_outcome']
model = sm.Logit(y, X).fit(disp=False)

# Build the clean summary
stats = pd.DataFrame({
    'feature': model.params.index,
    'coef':    model.params.values,
    'pval':    model.pvalues.values
})
# Drop intercept
stats = stats[stats['feature'] != 'const']

# First compute odds_ratio, then round everything
stats = (
    stats
      .assign(odds_ratio = np.exp(stats['coef']))          # compute OR
      .assign(                                             
          coef       = lambda df: df['coef'].round(3),     # round coef
          odds_ratio = lambda df: df['odds_ratio'].round(3),  # round OR
          pval       = lambda df: df['pval'].apply(lambda p: f"{p:.3g}")  # tidy pval
      )
      .sort_values('pval')                                 # sort by significance
      .reset_index(drop=True)
)

# Show only the columns you care about
print(stats[['feature','coef','odds_ratio','pval']])


                    feature   coef  odds_ratio      pval
0               diff_StrAcc  0.012       1.012  0.000367
1               diff_Weight  0.007       1.007   0.00558
2               diff_StrDef  0.008       1.008    0.0406
3                diff_Draws -0.074       0.929    0.0514
4                diff_Reach  0.015       1.015      0.17
5                diff_TDAcc -0.001       0.999     0.226
6               diff_SubAvg  0.035       1.036     0.315
7            diff_Height_in -0.007       0.993     0.637
8      diff_Stance_Southpaw  0.078       1.081         1
9   diff_Stance_Open_stance -0.201       0.818         1
10     diff_Stance_Orthodox  0.023       1.023         1
11       diff_Stance_Switch  0.100       1.105         1
12                diff_SLpM  0.243       1.275  1.08e-22
13               diff_TDAvg  0.081       1.084  2.07e-05
14               diff_TDDef  0.008       1.008  2.34e-11
15                diff_Wins  0.077       1.080  3.17e-75
16              diff_Losses -0.